In [1]:
import warnings
warnings.filterwarnings('ignore')

## Pivot
- 데이터 불러오기

In [21]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/pandas/main/under5MortalityRate.csv")
df.head(2)

,Location,Period,Indicator,Dim1,First Tooltip
0,Afghanistan,2019,Under-five mortality rate (probability of dyin...,Both sexes,60.27 [47.44-74.62]
1,Afghanistan,2019,Under-five mortality rate (probability of dyin...,Male,63.83 [50.04-79.16]


- Indicator 컬럼을 삭제하고 First Tooltip 컬럼에서 신뢰구간에 해당하는 표현을 지우기



In [22]:
df.drop('Indicator', axis=1, inplace=True)
df

,Location,Period,Dim1,First Tooltip
0,Afghanistan,2019,Both sexes,60.27 [47.44-74.62]
1,Afghanistan,2019,Male,63.83 [50.04-79.16]
2,Afghanistan,2019,Female,56.57 [44.54-70.18]
3,Afghanistan,2018,Both sexes,62.54 [50.64-75.41]
4,Afghanistan,2018,Male,66.08 [53.41-79.76]
...,...,...,...,...
29994,Timor-Leste,2002,Both sexes,96.97 [87.36-107.7]
29995,Timor-Leste,2002,Male,102.6 [92.14-114]
29996,Timor-Leste,2002,Female,91.04 [81.92-101.5]
29997,Timor-Leste,2001,Both sexes,102.5 [92.72-113.3]


In [25]:
df['First Tooltip'] = df['First Tooltip'].map(lambda x : float(x.split("[")[0]))
df.head(1)

,Location,Period,Dim1,First Tooltip
0,Afghanistan,2019,Both sexes,60.27


- 연도가 2015년 이상, Dim1이 Both sexes인 케이스만 추출하기

In [27]:
target = df[(df['Period'] >= 2015) & (df['Dim1'] == 'Both sexes')]
target.head()

,Location,Period,Dim1,First Tooltip
0,Afghanistan,2019,Both sexes,60.27
3,Afghanistan,2018,Both sexes,62.54
6,Afghanistan,2017,Both sexes,64.94
9,Afghanistan,2016,Both sexes,67.57
12,Afghanistan,2015,Both sexes,70.44


- 위에서 추출한 데이터로 나라에 따른 연도별 사망률을 데이터프레임화 하기

In [28]:
target_pivot = target.pivot(index='Location', columns='Period', values = 'First Tooltip')
target_pivot

Period,2015,2016,2017,2018,2019
Location,,,,,
Afghanistan,70.44,67.57,64.94,62.54,60.27
Albania,9.57,9.42,9.42,9.53,9.68
Algeria,25.18,24.79,24.32,23.81,23.26
Andorra,3.53,3.37,3.22,3.09,2.97
Angola,88.20,84.21,80.62,77.67,74.69
...,...,...,...,...,...
Syrian Arab Republic,23.18,23.27,22.97,22.11,21.53
Tajikistan,37.75,36.82,35.81,34.80,33.78
Thailand,10.80,10.32,9.86,9.42,9.01


- Dim1에 따른 연도별 사망비율의 평균 구하기

In [33]:
# df.groupby(['Dim1', 'Period', 'First Tooltip']).mean()
df_1 = df.pivot_table(index='Dim1', columns='Period', values='First Tooltip', aggfunc='mean')
df_1

Period,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Dim1,,,,,,,,,,,,,,,,,,,,,
Both sexes,147.700816,155.537544,157.811094,156.147206,154.539857,155.797179,159.241163,156.170114,150.813222,150.574000,...,37.718488,35.573663,34.290988,33.099360,32.053314,31.012093,29.956337,29.030465,28.083837,27.191744
Female,140.909796,149.210175,151.516094,150.250882,148.688286,149.843205,153.048721,149.988295,144.719667,144.451474,...,34.953023,32.877616,31.654070,30.521337,29.524302,28.544360,27.542035,26.675291,25.782616,24.945349
Male,154.151224,161.538246,163.760781,161.742059,160.081000,161.456923,165.089535,162.015000,156.573556,156.375053,...,40.340174,38.140291,36.793081,35.543663,34.446105,33.354302,32.242616,31.273198,30.283023,29.350349


- 다른 데이터 불러오기

In [34]:
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/pandas/main/winter.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5770 entries, 0 to 5769
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        5770 non-null   int64 
 1   City        5770 non-null   object
 2   Sport       5770 non-null   object
 3   Discipline  5770 non-null   object
 4   Athlete     5770 non-null   object
 5   Country     5770 non-null   object
 6   Gender      5770 non-null   object
 7   Event       5770 non-null   object
 8   Medal       5770 non-null   object
dtypes: int64(1), object(8)
memory usage: 405.8+ KB


- 데이터에서 한국 KOR 데이터만 추출하기

In [36]:
kor = data[data['Country'] == 'KOR']
kor

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
2652,1992,Albertville,Skating,Short Track Speed Skating,"LEE, Jun-Ho",KOR,Men,1000M,Bronze
2653,1992,Albertville,Skating,Short Track Speed Skating,"KIM, Ki-Hoon",KOR,Men,1000M,Gold
2671,1992,Albertville,Skating,Short Track Speed Skating,"KIM, Ki-Hoon",KOR,Men,5000M Relay,Gold
2672,1992,Albertville,Skating,Short Track Speed Skating,"LEE, Jun-Ho",KOR,Men,5000M Relay,Gold
2673,1992,Albertville,Skating,Short Track Speed Skating,"MO, Ji-Soo",KOR,Men,5000M Relay,Gold
...,...,...,...,...,...,...,...,...,...
5528,2014,Sochi,Skating,Short Track Speed Skating,"PARK, Seung-Hi",KOR,Women,500M,Bronze
5544,2014,Sochi,Skating,Speed skating,"JOO, Hyong Jun",KOR,Men,Team Pursuit,Silver
5545,2014,Sochi,Skating,Speed skating,"KIM, Cheol Min",KOR,Men,Team Pursuit,Silver
5546,2014,Sochi,Skating,Speed skating,"LEE, Seung Hoon",KOR,Men,Team Pursuit,Silver


- 한국 올림픽 메달리스트 데이터에서 년도에 따른 medal갯수를 데이터프레임화 하기

In [39]:
kor.pivot_table(index='Year', columns='Medal', aggfunc='size').fillna(0)

Medal,Bronze,Gold,Silver
Year,,,
1992,1.0,5.0,1.0
1994,1.0,8.0,1.0
1998,2.0,6.0,4.0
2002,0.0,5.0,2.0
2006,2.0,14.0,3.0
2010,2.0,6.0,10.0
2014,2.0,7.0,5.0


- 전체 데이터에서 sport 종류에 따른 성별수 구하기

In [42]:
data.pivot_table(index='Sport', columns = 'Gender', aggfunc='size')

Gender,Men,Women
Sport,,
Biathlon,270,150
Bobsleigh,416,36
Curling,97,75
Ice Hockey,1231,305
Luge,135,45
Skating,665,564
Skiing,1130,651


- 전체 데이터에서 Discipline 종류에 따른 Medal 수 구하기

In [43]:
data.pivot_table(index='Discipline', columns='Medal', aggfunc='size')

Medal,Bronze,Gold,Silver
Discipline,,,
Alpine Skiing,141,143,144
Biathlon,139,140,141
Bobsleigh,147,134,141
Cross Country Skiing,263,264,262
Curling,56,58,58
Figure skating,118,122,119
Freestyle Skiing,34,34,34
Ice Hockey,512,510,514
Luge,60,62,58


## Merge, Concat
- 데이터 불러오기

In [44]:
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/mergeTEst.csv',index_col= 0)
test.head(1)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Afghanistan,64.023,61.64,59.367,57.17,55.08,53.107,51.267,49.56,47.983,46.453


In [45]:
df1 = test.iloc[:4, :]
df2 = test.iloc[4:, :]

In [49]:
display(df1)
display(df2)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Afghanistan,64.023,61.640,59.367,57.170,55.08,53.107,51.267,49.560,47.983,46.453
Albania,11.803,10.807,9.943,9.267,8.79,8.493,8.363,8.363,8.453,8.597
Algeria,23.540,22.907,22.450,22.117,21.85,21.587,21.257,20.850,20.407,19.930
Andorra,4.240,4.033,3.843,3.667,3.49,3.330,3.187,3.060,2.933,2.827


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Angola,75.713,71.280,67.233,63.570,60.430,57.757,55.510,53.460,51.757,50.093
Antigua and Barbuda,8.667,8.223,7.807,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,12.887,12.380,11.840,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,16.497,15.677,14.897,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,3.993,3.803,3.623,3.467,3.343,3.253,3.183,3.137,3.090,3.047
Austria,3.573,3.463,3.333,3.210,3.113,3.043,2.987,2.943,2.897,2.843


- df1, df2 합치기

In [50]:
tot1 = pd.concat([df1, df2])
tot1

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Afghanistan,64.023,61.640,59.367,57.170,55.080,53.107,51.267,49.560,47.983,46.453
Albania,11.803,10.807,9.943,9.267,8.790,8.493,8.363,8.363,8.453,8.597
Algeria,23.540,22.907,22.450,22.117,21.850,21.587,21.257,20.850,20.407,19.930
Andorra,4.240,4.033,3.843,3.667,3.490,3.330,3.187,3.060,2.933,2.827
Angola,75.713,71.280,67.233,63.570,60.430,57.757,55.510,53.460,51.757,50.093
Antigua and Barbuda,8.667,8.223,7.807,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,12.887,12.380,11.840,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,16.497,15.677,14.897,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,3.993,3.803,3.623,3.467,3.343,3.253,3.183,3.137,3.090,3.047


- df3과 df4 합치기(공통 연도만 고려)

In [51]:
df3 = test.iloc[:2,:4]
df4 = test.iloc[5:,3:]

display(df3)
display(df4)

,2010,2011,2012,2013
Location,,,,
Afghanistan,64.023,61.640,59.367,57.170
Albania,11.803,10.807,9.943,9.267


,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,
Antigua and Barbuda,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,3.467,3.343,3.253,3.183,3.137,3.090,3.047
Austria,3.210,3.113,3.043,2.987,2.943,2.897,2.843


In [53]:
tot2 = pd.concat([df3, df4], join='inner')
tot2

,2013
Location,
Afghanistan,57.170
Albania,9.267
Antigua and Barbuda,7.420
Argentina,11.283
Armenia,14.170
Australia,3.467
Austria,3.210


- df3과 df4 합치기(모든 컬럼 포함, 결측치는 0)

In [55]:
tot3 = pd.concat([df3, df4], join='outer').fillna(0)
tot3

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Afghanistan,64.023,61.640,59.367,57.170,0.000,0.000,0.000,0.000,0.000,0.000
Albania,11.803,10.807,9.943,9.267,0.000,0.000,0.000,0.000,0.000,0.000
Antigua and Barbuda,0.000,0.000,0.000,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,0.000,0.000,0.000,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,0.000,0.000,0.000,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,0.000,0.000,0.000,3.467,3.343,3.253,3.183,3.137,3.090,3.047
Austria,0.000,0.000,0.000,3.210,3.113,3.043,2.987,2.943,2.897,2.843


In [56]:
df5 = test.T.iloc[:7,:3]
df6 = test.T.iloc[6:,2:5]

display(df5)
display(df6)

Location,Afghanistan,Albania,Algeria
2010,64.023,11.803,23.540
2011,61.640,10.807,22.907
2012,59.367,9.943,22.450
2013,57.170,9.267,22.117
2014,55.080,8.790,21.850
2015,53.107,8.493,21.587
2016,51.267,8.363,21.257


Location,Algeria,Andorra,Angola
2016,21.257,3.187,55.510
2017,20.850,3.060,53.460
2018,20.407,2.933,51.757
2019,19.930,2.827,50.093


- df5, df6 합치기(merge이용, Algeria컬럼을 key로 하고 두 데이터를 모두 포함하는 데이터만 출력)

In [57]:
tot4 = pd.merge(df5, df6, on = 'Algeria', how='inner')
tot4

Location,Afghanistan,Albania,Algeria,Andorra,Angola
0,51.267,8.363,21.257,3.187,55.51


In [58]:
tot5 = pd.merge(df5, df6, on = 'Algeria', how='outer')
tot5

Location,Afghanistan,Albania,Algeria,Andorra,Angola
0,64.023,11.803,23.540,NaN,NaN
1,61.640,10.807,22.907,NaN,NaN
2,59.367,9.943,22.450,NaN,NaN
3,57.170,9.267,22.117,NaN,NaN
4,55.080,8.790,21.850,NaN,NaN
5,53.107,8.493,21.587,NaN,NaN
6,51.267,8.363,21.257,3.187,55.510
7,NaN,NaN,20.850,3.060,53.460
8,NaN,NaN,20.407,2.933,51.757
9,NaN,NaN,19.930,2.827,50.093
